In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks, knockout_FBA_simple

from functools import partial
from src.FBA_scripts.met_task_functions import constrain_model, read_tasks


In [2]:
# Single model
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/pancreas.xml'
#model = cobra.io.read_sbml_model(model_file_path)

In [3]:
# Models for tasks
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpqhpj5l9q.lp
Reading time = 0.07 seconds
: 5597 rows, 15340 columns, 67174 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpdq6wlt_7.lp
Reading time = 0.07 seconds
: 5597 rows, 15340 columns, 67174 nonzeros


In [85]:
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
# Constrain model
task = task_list[-1]
for subset in [task[0], task[1]]:
    for rx in subset:
        if rx == 'ALLMETSIN':
            # Adds boundary metabolites for other reactions when ALLMETSIN is used
            for r in subset[1:]:
                for m2 in r.metabolites:
                    for r2 in m2.reactions:
                        if r2.boundary and r2.id != r.id:
                            # Could also just remove the reactions, or set them 0, 0
                            r2.add_metabolites({Metabolite(
                                                m2.id[:-4] + 'x[x]',
                                                formula=m2.formula,
                                                name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                compartment='x'): 1})
            continue
        model_list[3].add_reaction(rx)


In [16]:
# FBA + knockouts

res_list = []

In [ ]:
model = model_list[0]
res_list.append(round(knockout_FBA_simple(model, []), 3))
res_list.append(round(knockout_FBA_simple(model, ['ENSG00000134538']), 3))

In [114]:
model = model_list[3]
with model:
    # Bilirubin sink
    model.reactions.get_by_id('HMR_9273').lower_bound = -1000
    model.reactions.get_by_id('HMR_9273').upper_bound = -249

    # Bilirubin-Bisglucuronoside sink
    model.reactions.get_by_id('HMR_9271').lower_bound = 0
    model.reactions.get_by_id('HMR_9271').upper_bound = 0

    # Bilirubin-monoglucuronoside sink
    model.reactions.get_by_id('HMR_9272').lower_bound = 10
    model.reactions.get_by_id('HMR_9272').upper_bound = 1000

    # Heme synthesis
    model.reactions.get_by_id('HMR_4758').lower_bound = 0
    model.reactions.get_by_id('HMR_4758').upper_bound = 1

    # Oxygen
    model.reactions.get_by_id('HMR_9048').lower_bound = -10
    # Glucose
    model.reactions.get_by_id('HMR_9034').lower_bound = -6

    # Threonine
    model.reactions.get_by_id('HMR_9044').lower_bound = -1

    #model.objective = 'HMR_9271'
    #model.objective_direction = 'max'

    print(knockout_FBA_simple(model, []))
    print(knockout_FBA_simple(model, ['ENSG00000134538', 'ENSG00000184999', 'ENSG00000103222']))

2.1839945137303576
2.1839945137303576


In [7]:
res_list_lung = res_list

[82.17122442635676, 81.6253209998778]


In [18]:
print('Lung results: ' + str(res_list_lung))
print(res_list)


Lung results: [82.17122442635676, 81.6253209998778]
[111.339, 111.339]


In [117]:
# FBA + knockouts

res_list = []

In [72]:
tissue = 'small_intestine'

model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpeuv8erle.lp
Reading time = 0.07 seconds
: 5811 rows, 16420 columns, 71822 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpk4d01zor.lp
Reading time = 0.07 seconds
: 5811 rows, 16420 columns, 71822 nonzeros


In [73]:
if True:
    genes = [gene.id for gene in model_list[0].genes]
else:
    genes = pd.read_table('C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/{0}_non_essential.tsv'.format(tissue), index_col=0)['gene_ids'].tolist()

# Create and run combinations for individual samples
t='ENSG00000177000;ENSG00000159640;ENSG00000188906;ENSG00000085563;ENSG00000084207;ENSG00000159640;ENSG00000140718;ENSG00000111012;ENSG00000091831;ENSG00000087237'

input = t.split(';')
print('Num genes: ' + str(len(input)))
input = list(set(input).intersection(genes))

combs = []
for i in range(1, 5): #len(input) + 1):
    for c in itertools.combinations(input, i):
        combs.append(list(c))

combs.append(input)
test_df = pd.DataFrame({'gene_ids': combs})

print('Num combs: ' + str(len(combs)))

task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks_min_ess_aa.tsv', model_list)
# Adds empty task result list column
test_df['tasks_results'] = np.empty((len(test_df), 0)).tolist()
test_df.reset_index(inplace=True)
test_df.rename(columns={'index': 'sample_ids'}, inplace=True)
test_df['sample_ids'] = test_df['sample_ids'].apply(lambda x: x + 1)

# Actual FBA
print(test_df.shape[0])
#print(test_df)

Num genes: 10
Num combs: 31
31


In [74]:
start_time = time.time()

results = parallelize_dataframe(test_df, partial(knockout_FBA_w_tasks, task_list, model_list), n_cores=2)

end_time = time.time()
print('Total time: %.6f seconds' % (end_time - start_time))

results.loc[-1] = ['REF', [], round(model_list[0].slim_optimize(error_value='nan'), 3), [1]]

# Put REF on top
results.index = results.index + 1  # shifting index
results.sort_index(inplace=True)

results['gene_ids'] = results['gene_ids'].apply(';'.join)
results['tasks_results'] = results['tasks_results'].apply(lambda x: x if not all(x) else ['ALL PASS'])

results.reset_index(inplace=True, drop=True)
results[['sample_ids', 'gene_ids', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/phewas_results/FBA_selected/result_test_MTHFR_ex_{0}.tsv'.format(tissue), sep='\t')

Total time: 78.425046 seconds


In [8]:
gene_list = 'ENSG00000113407;ENSG00000116039;ENSG00000198130;ENSG00000206077;ENSG00000034152'.split(';')
for gene in gene_list:
    print(gene + ': ' + str(knockout_FBA_simple(model_list[0], [gene])))




ENSG00000113407: 0.0
ENSG00000116039: 75.00989279272777
ENSG00000198130: 75.00989279272777
ENSG00000206077: 75.00989279272777
ENSG00000034152: 75.00989279272777
